In [2]:
%tensorflow_version 1.x

import cv2
import json

import numpy as np
import pandas as pd

from functools import partial
from pathlib import Path 
from tqdm import tqdm

from google.colab.patches import cv2_imshow

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

import keras
from keras.applications import VGG16, InceptionV3
from keras.layers.core import Dropout
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers import Input
from keras.models import Model, load_model, save_model
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau,LearningRateScheduler, EarlyStopping
from keras import backend as K
from keras.layers.normalization import BatchNormalization


import os
from google.colab import drive

from IPython.core.display import display

from sklearn.model_selection import train_test_split

import shutil




Using TensorFlow backend.


In [3]:
drive_mount = os.path.abspath('/content/gdrive')
base_dir = os.path.join(drive_mount, 'My Drive/human_data')
models_dir = os.path.join(base_dir, 'saved_models')


# mount gdrive and unzip data

drive.mount(drive_mount, force_remount=True)
!unzip -qo "./gdrive/My Drive/human_data/hvc_data.zip"
print(os.listdir(base_dir))
%ls

Mounted at /content/gdrive
['hvc_data.zip', 'saved_models']
gdrive/  hvc_annotations.csv  resized/  sample_data/


In [4]:
# load annotations
df = pd.read_csv("hvc_annotations.csv")
del df["filename"] # remove unwanted column
display(df)

,gender,imagequality,age,weight,carryingbag,footwear,emotion,bodypose,image_path
0,male,Average,35-45,normal-healthy,Grocery/Home/Plastic Bag,Normal,Neutral,Front-Frontish,resized/1.jpg
1,female,Average,35-45,over-weight,None,Normal,Angry/Serious,Front-Frontish,resized/2.jpg
2,male,Good,45-55,normal-healthy,Grocery/Home/Plastic Bag,CantSee,Neutral,Front-Frontish,resized/3.jpg
3,male,Good,45-55,normal-healthy,Daily/Office/Work Bag,Normal,Neutral,Front-Frontish,resized/4.jpg
4,female,Good,35-45,slightly-overweight,None,CantSee,Neutral,Front-Frontish,resized/5.jpg
...,...,...,...,...,...,...,...,...,...
13568,male,Average,35-45,slightly-overweight,None,Normal,Happy,Front-Frontish,resized/13570.jpg
13569,female,Average,25-35,normal-healthy,Daily/Office/Work Bag,Fancy,Neutral,Front-Frontish,resized/13571.jpg
13570,female,Bad,35-45,normal-healthy,Grocery/Home/Plastic Bag,Normal,Neutral,Side,resized/13572.jpg
13571,female,Bad,25-35,over-weight,Daily/Office/Work Bag,Normal,Neutral,Front-Frontish,resized/13573.jpg


In [5]:
# one hot encoding of labels

one_hot_df = pd.concat(
    [
        df[["image_path"]],
        pd.get_dummies(df.gender, prefix="gender"),
        pd.get_dummies(df.imagequality, prefix="imagequality"),
        pd.get_dummies(df.age, prefix="age"),
        pd.get_dummies(df.weight, prefix="weight"),
        pd.get_dummies(df.carryingbag, prefix="carryingbag"),
        pd.get_dummies(df.footwear, prefix="footwear"),
        pd.get_dummies(df.emotion, prefix="emotion"),
        pd.get_dummies(df.bodypose, prefix="bodypose"),
    ], 
    axis = 1
)

display(one_hot_df)


,image_path,gender_female,gender_male,imagequality_Average,imagequality_Bad,imagequality_Good,age_15-25,age_25-35,age_35-45,age_45-55,age_55+,weight_normal-healthy,weight_over-weight,weight_slightly-overweight,weight_underweight,carryingbag_Daily/Office/Work Bag,carryingbag_Grocery/Home/Plastic Bag,carryingbag_None,footwear_CantSee,footwear_Fancy,footwear_Normal,emotion_Angry/Serious,emotion_Happy,emotion_Neutral,emotion_Sad,bodypose_Back,bodypose_Front-Frontish,bodypose_Side
0,resized/1.jpg,0,1,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0
1,resized/2.jpg,1,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,1,1,0,0,0,0,1,0
2,resized/3.jpg,0,1,0,0,1,0,0,0,1,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,1,0
3,resized/4.jpg,0,1,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0
4,resized/5.jpg,1,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,1,0,0,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13568,resized/13570.jpg,0,1,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,1,0
13569,resized/13571.jpg,1,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,1,0
13570,resized/13572.jpg,1,0,0,1,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1
13571,resized/13573.jpg,1,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0


In [0]:
# Label columns per attribute
_gender_cols_ = [col for col in one_hot_df.columns if col.startswith("gender")]
_imagequality_cols_ = [col for col in one_hot_df.columns if col.startswith("imagequality")]
_age_cols_ = [col for col in one_hot_df.columns if col.startswith("age")]
_weight_cols_ = [col for col in one_hot_df.columns if col.startswith("weight")]
_carryingbag_cols_ = [col for col in one_hot_df.columns if col.startswith("carryingbag")]
_footwear_cols_ = [col for col in one_hot_df.columns if col.startswith("footwear")]
_emotion_cols_ = [col for col in one_hot_df.columns if col.startswith("emotion")]
_bodypose_cols_ = [col for col in one_hot_df.columns if col.startswith("bodypose")]

In [0]:
col_list = [_gender_cols_, _imagequality_cols_, _age_cols_, _weight_cols_, _carryingbag_cols_, _footwear_cols_, _emotion_cols_, _bodypose_cols_]

In [8]:
for entity in (col_list):
  print(entity)

['gender_female', 'gender_male']
['imagequality_Average', 'imagequality_Bad', 'imagequality_Good']
['age_15-25', 'age_25-35', 'age_35-45', 'age_45-55', 'age_55+']
['weight_normal-healthy', 'weight_over-weight', 'weight_slightly-overweight', 'weight_underweight']
['carryingbag_Daily/Office/Work Bag', 'carryingbag_Grocery/Home/Plastic Bag', 'carryingbag_None']
['footwear_CantSee', 'footwear_Fancy', 'footwear_Normal']
['emotion_Angry/Serious', 'emotion_Happy', 'emotion_Neutral', 'emotion_Sad']
['bodypose_Back', 'bodypose_Front-Frontish', 'bodypose_Side']


In [0]:
class PersonDataGenerator(keras.utils.Sequence):
    """Ground truth data generator"""

    def __init__(self, df, batch_size=32, shuffle=True, augmentation=None):
        self.df = df
        self.batch_size=batch_size
        self.shuffle = shuffle
        self.on_epoch_end()
        self.augmentation = augmentation

    def __len__(self):
        return int(np.floor(self.df.shape[0] / self.batch_size))

    def __getitem__(self, index):
        """fetch batched images and targets"""
        batch_slice = slice(index * self.batch_size, (index + 1) * self.batch_size)
        items = self.df.iloc[batch_slice]
        
        images = np.stack([cv2.imread(item["image_path"]) for _, item in items.iterrows()])    
        
        if self.augmentation:
          eraser = get_random_eraser()
          aug_image_stack = np.stack([eraser(img) for img in images])    

        # if self.augmentation is not None:
        #     images = self.augmentation.flow(images, shuffle=False).next()
        
        target = {
            "gender_output": items[_gender_cols_].values,
            "image_quality_output": items[_imagequality_cols_].values,
            "age_output": items[_age_cols_].values,
            "weight_output": items[_weight_cols_].values,
            "bag_output": items[_carryingbag_cols_].values,
            "pose_output": items[_bodypose_cols_].values,
            "footwear_output": items[_footwear_cols_].values,
            "emotion_output": items[_emotion_cols_].values,
        }

        if self.augmentation:
          return aug_image_stack, target

        return images, target

    def on_epoch_end(self):
        """Updates indexes after each epoch"""
        if self.shuffle == True:
            self.df = self.df.sample(frac=1).reset_index(drop=True)

In [10]:

train_df, val_df = train_test_split(one_hot_df, test_size=0.15)

print(train_df.shape, val_df.shape)
display(train_df)


(11537, 28) (2036, 28)


,image_path,gender_female,gender_male,imagequality_Average,imagequality_Bad,imagequality_Good,age_15-25,age_25-35,age_35-45,age_45-55,age_55+,weight_normal-healthy,weight_over-weight,weight_slightly-overweight,weight_underweight,carryingbag_Daily/Office/Work Bag,carryingbag_Grocery/Home/Plastic Bag,carryingbag_None,footwear_CantSee,footwear_Fancy,footwear_Normal,emotion_Angry/Serious,emotion_Happy,emotion_Neutral,emotion_Sad,bodypose_Back,bodypose_Front-Frontish,bodypose_Side
9257,resized/9258.jpg,1,0,0,1,0,0,0,0,1,0,0,0,0,1,1,0,0,0,1,0,0,0,1,0,0,1,0
2661,resized/2662.jpg,1,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,1,0,0,0,1,0
7031,resized/7032.jpg,0,1,0,1,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1,0,1,0,0
12162,resized/12164.jpg,0,1,0,0,1,0,1,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,1,0,0,1,0
1284,resized/1285.jpg,1,0,1,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2694,resized/2695.jpg,1,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,1
3724,resized/3725.jpg,0,1,1,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,1,0
10971,resized/10973.jpg,0,1,1,0,0,0,0,1,0,0,0,0,1,0,1,0,0,1,0,0,1,0,0,0,0,1,0
1558,resized/1559.jpg,0,1,1,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,1,0,0,0,1


In [0]:
def get_random_eraser(p=0.5, s_l=0.02, s_h=0.4, r_1=0.3, r_2=1/0.3, v_l=0, v_h=255, pixel_level=False):
    def eraser(input_img):
        img_h, img_w, _ = input_img.shape
        p_1 = np.random.rand()

        if p_1 > p:
            return input_img

        while True:
            s = np.random.uniform(s_l, s_h) * img_h * img_w
            r = np.random.uniform(r_1, r_2)
            w = int(np.sqrt(s / r))
            h = int(np.sqrt(s * r))
            left = np.random.randint(0, img_w)
            top = np.random.randint(0, img_h)

            if left + w <= img_w and top + h <= img_h:
                break

        c = np.random.uniform(v_l, v_h)
        input_img[top:top + h, left:left + w, :] = c

        return input_img

    return eraser

In [0]:
datagen = ImageDataGenerator(preprocessing_function=get_random_eraser(v_l=0, v_h=1, pixel_level=False))

In [0]:
# create train and validation data generators
train_gen = PersonDataGenerator(train_df, batch_size=32,augmentation=datagen)
valid_gen = PersonDataGenerator(val_df, batch_size=64, shuffle=False,augmentation=False)

In [14]:
# get number of output units from data
images, targets = next(iter(train_gen))
num_units = { k.split("_output")[0]:v.shape[1] for k, v in targets.items()}
print('no. of units:', num_units)

no. of units: {'gender': 2, 'image_quality': 3, 'age': 5, 'weight': 4, 'bag': 3, 'pose': 3, 'footwear': 3, 'emotion': 4}


In [15]:
backbone = VGG16(
    weights="imagenet", 
    include_top=False, 
    input_tensor=Input(shape=(224, 224, 3))
)

neck = backbone.output
neck = BatchNormalization()(neck)
neck = Flatten(name="flatten")(neck)
neck = Dense(256, activation="relu")(neck)

def build_tower(in_layer):
    neck = Dropout(0.05)(in_layer)
    neck = BatchNormalization()(neck)
    neck = Dense(128, activation="relu")(neck)
    neck = Dropout(0.1)(neck)
    neck = BatchNormalization()(neck)
    neck = Dense(128, activation="relu")(neck)
    return neck


def build_head(name, in_layer):
    return Dense(
        num_units[name], activation="softmax", name=f"{name}_output")(in_layer)

# heads
gender = build_head("gender", build_tower(neck))
image_quality = build_head("image_quality", build_tower(neck))
age = build_head("age", build_tower(neck))
weight = build_head("weight", build_tower(neck))
bag = build_head("bag", build_tower(neck))
footwear = build_head("footwear", build_tower(neck))
emotion = build_head("emotion", build_tower(neck))
pose = build_head("pose", build_tower(neck))


model = Model(
    inputs=backbone.input, 
    outputs=[gender, image_quality, age, weight, bag, footwear, pose, emotion]
)













Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [16]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 224, 224, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 224, 224, 64) 36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 112, 112, 64) 0           block1_conv2[0][0]               
____________________________________________________________________________________________

In [0]:
# freeze backbone
for layer in backbone.layers:
	layer.trainable = False

In [18]:

losses = {
	"gender_output": "binary_crossentropy",
	"image_quality_output": "categorical_crossentropy",
	"age_output": "categorical_crossentropy",
	"weight_output": "categorical_crossentropy",
  "bag_output": "categorical_crossentropy",
  "footwear_output": "categorical_crossentropy",
  "pose_output": "categorical_crossentropy",
  "emotion_output": "categorical_crossentropy"
}
loss_weights = {"gender_output": 1.0, "image_quality_output": 1.0, "age_output": 1.3, "weight_output":1.0,"bag_output":1.0,"footwear_output":1.0,"pose_output":1.0,"emotion_output":1.0}
opt = SGD(lr=1e-3, momentum=0.9)
model.compile(
    optimizer=opt,
    loss= losses,
    #loss="categorical_crossentropy", 
    loss_weights=loss_weights, 
    metrics=["accuracy"]
)



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 45:
        lr *= 0.5e-3
    elif epoch > 35:
        lr *= 1e-3
    elif epoch > 25:
        lr *= 1e-2
    elif epoch > 15:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

In [0]:
# def update_callbacks():

#   # Prepare model model saving directory.
#   # save_dir = os.path.join(os.getcwd(), 'saved_models')
#   save_dir = F"/content/gdrive/My Drive/saved_models" 
#   model_name = 'Wk5_model.{epoch:03d}.{val_acc:.4f}.h5'
#   if not os.path.isdir(save_dir):
#       os.makedirs(save_dir)
#   filepath = os.path.join(save_dir, model_name)

#   # Prepare callbacks for model saving and for learning rate adjustment.
#   checkpoint = ModelCheckpoint(filepath=filepath,
#                               monitor='val_acc',
#                               verbose=1,
#                               save_best_only=True)

#   lr_scheduler = LearningRateScheduler(lr_schedule)

#   lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
#                                 cooldown=0,
#                                 patience=5,
#                                 min_lr=0.5e-6)

#   #callbacks = [checkpoint, lr_reducer, lr_scheduler]

#   callbacks = [lr_reducer,lr_scheduler]

#   return callbacks

In [21]:
import datetime
# Prepare model model saving directory.
# save_dir = os.path.join(os.getcwd(), 'saved_models')
t = datetime.datetime.today()
print(str(datetime.datetime.today()))
save_dir = "/content/gdrive/My Drive/Colab Notebooks/Assignment5/saved_models/"+str(t)
model_name = 'A5_model.{epoch:03d}.h5'
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)
filepath

2019-12-29 15:45:25.128006


'/content/gdrive/My Drive/Colab Notebooks/Assignment5/saved_models/2019-12-29 15:45:25.127965/A5_model.{epoch:03d}.h5'

In [0]:
# Prepare callbacks for model saving and for learning rate adjustment.
checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='val_loss',
                             verbose=1,
                             save_best_only=True)
''

#es = EarlyStopping(monitor='val_loss', verbose=1, patience=5, mode='min')

lr_scheduler = LearningRateScheduler(lr_schedule, verbose=1)

# lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
#                                cooldown=0,
#                                patience=5,
#                                min_lr=0.5e-6)

# callbacks = [checkpoint, lr_reducer, lr_scheduler]
callbacks = [checkpoint,lr_scheduler]

In [29]:
model.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    callbacks = callbacks,
    workers=6, 
    epochs=100,
    verbose=1
)



Epoch 1/100
Learning rate:  0.001

Epoch 00001: LearningRateScheduler setting learning rate to 0.001.
360/360 [==============================] - 61s 170ms/step - loss: 8.1582 - gender_output_loss: 0.5158 - image_quality_output_loss: 0.9920 - age_output_loss: 1.5100 - weight_output_loss: 1.0985 - bag_output_loss: 0.9635 - footwear_output_loss: 0.8673 - pose_output_loss: 0.7655 - emotion_output_loss: 0.9926 - gender_output_acc: 0.7479 - image_quality_output_acc: 0.5246 - age_output_acc: 0.3558 - weight_output_acc: 0.5802 - bag_output_acc: 0.5541 - footwear_output_acc: 0.6082 - pose_output_acc: 0.6708 - emotion_output_acc: 0.6779 - val_loss: 7.4500 - val_gender_output_loss: 0.4060 - val_image_quality_output_loss: 0.8998 - val_age_output_loss: 1.4325 - val_weight_output_loss: 1.0264 - val_bag_output_loss: 0.9063 - val_footwear_output_loss: 0.7978 - val_pose_output_loss: 0.6008 - val_emotion_output_loss: 0.9506 - val_gender_output_acc: 0.8160 - val_image_quality_output_acc: 0.5766 - val_a

In [30]:
results = model.evaluate_generator(valid_gen, verbose=1)

31/31 [==============================] - 5s 174ms/step


In [31]:
results

[11.052993097612935,
 0.5063016563653946,
 1.4301760408186144,
 2.350307191571882,
 1.5013189777251212,
 1.2914905125094998,
 1.1030988693237305,
 0.7409410813162404,
 1.4242667478899802,
 0.827116935483871,
 0.5216733870967742,
 0.35735887096774194,
 0.5569556451612904,
 0.5871975806451613,
 0.6456653225806451,
 0.7792338709677419,
 0.6391129032258065]